In [2]:
import pandas as pd
import torch
from matplotlib import pyplot as plt

In [3]:
!pip3 install sentence-transformers
!pip3 install matplotlib


[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: python3.11 -m pip install --upgrade pip

[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: python3.11 -m pip install --upgrade pip


In [4]:
from sentence_transformers import SentenceTransformer, util

/opt/homebrew/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
# load responses
responses_df = pd.read_csv('./GPT responses.csv')
responses_df

,Full Text,Relevant Phrase,Predicate,ID subgroups,Annotator,ChatGPT response,Extraction Outcome,prompt
0,"LGBTQ+ people are abnormal, crazy, have family...",are abnormal,abnormal,NaN,Katy,have health issues,correct,"From the following sentence, extract a stateme..."
1,"LGBTQ+ people are abnormal, crazy, have family...",crazy,crazy,NaN,Katy,are abnormal,correct,"From the following sentence, extract a stateme..."
2,"LGBTQ+ people are abnormal, crazy, have family...",have family issues,have family issues,NaN,Katy,have family issues,correct,"From the following sentence, extract a stateme..."
3,"LGBTQ+ people are abnormal, crazy, have family...",have health issues,have health issues,NaN,Katy,are crazy,correct,"From the following sentence, extract a stateme..."
4,Unfit for professional work,Unfit for professional work,unfit for professional work,NaN,Katy,struggle with mental health,hallucination,"From the following sentence, extract a stateme..."
...,...,...,...,...,...,...,...,...
273,slurs/ being treated like there's something wr...,NaN,there is something wrong,Pansexual,Katy,All pansexual people are abnormal.,grammar,"From the following sentence, extract a stateme..."
274,Being undermined in an academic environment,NaN,undermined in an academic environment,Pansexual,Katy,All pansexual people struggle.,hallucination,"From the following sentence, extract a stateme..."
275,"More willing to have threesomes, ""easy"", bigge...",NaN,more willing to have threesomes,Pansexual,Katy,engage in casual sex,hallucination,"From the following sentence, extract a stateme..."
276,"More willing to have threesomes, ""easy"", bigge...",NaN,easy,Pansexual,Katy,cheat on their partners,correct,"From the following sentence, extract a stateme..."


In [6]:
results_df = pd.DataFrame(columns=['Full Text', 'Predicate', 'ID subgroups', 'Best response', 'Exact Match', 'Phrase Cosine Sim.', 'Sent. Cosine Sim.', 'Extraction Outcome'])


In [7]:
embed_model = SentenceTransformer('sentence-transformers/all-mpnet-base-v2')

In [8]:
# NB: efficiency could be improved here with groupby and a lambda function, but the data is small enough it shouldn't matter too much

# find closest predicate for each response 
for index, row in responses_df.iterrows():
    # lowercase, strip periods and extra whitespace
    pred = row['Predicate'].lower().replace(".", '').strip()
    # get all correct predicates for this sentence
    responses = responses_df[responses_df['Full Text'] == row['Full Text']]['ChatGPT response'].unique().tolist()
    # lowercase, strip periods and extra whitespace
    for r in responses:
        r = r.lower().replace(".", '').strip()
    
    # ChatGPT response that best matches this predicate
    best_response = ''
    exact = 0
    # check exact matches
    if pred in responses:
        exact = 1
    
    # calculate SBERT cosine similarity 
    pred_embed = embed_model.encode([pred])
    resp_embed = embed_model.encode(responses)
    
    cosine_scores = util.cos_sim(pred_embed, resp_embed)
    # get max cosine score and its index
    best_index = torch.argmax(cosine_scores)
    best_response = responses[best_index]
    best_score = cosine_scores[0][best_index].item()
    
    # calculate SBERT cosine similarity for dummy sentence
    sent_pred = "All LGBTQ people are " + pred
    sent_responses = ["All LGBTQ people are " + r for r in responses]
    sp_embed = embed_model.encode([sent_pred])
    sr_embed = embed_model.encode(sent_responses)
    
    s_cosine_scores = util.cos_sim(sp_embed, sr_embed)
    # get max cosine score and its index
    s_best_index = torch.argmax(cosine_scores)
    if s_best_index != best_index:
        print("difference in best match!")
    s_best_response = responses[s_best_index]
    s_best_score = s_cosine_scores[0][s_best_index].item()
    
    # write row to results_df
    results_df.at[index, 'Full Text'] = row['Full Text']
    results_df.at[index, 'Predicate'] = row['Predicate']
    results_df.at[index, 'ID subgroups'] = row['ID subgroups'] if row.notnull()['ID subgroups'] else "LGBTQ"
    results_df.at[index, 'Best response'] = best_response
    results_df.at[index, 'Exact Match'] = exact
    results_df.at[index, 'Phrase Cosine Sim.'] = best_score
    results_df.at[index, 'Sent. Cosine Sim.'] = s_best_score
    results_df.at[index, 'Extraction Outcome'] = row['Extraction Outcome']
    
    print("wrote row", index)

wrote row 0
wrote row 1
wrote row 2
wrote row 3
wrote row 4
wrote row 5
wrote row 6
wrote row 7
wrote row 8
wrote row 9
wrote row 10
wrote row 11
wrote row 12
wrote row 13
wrote row 14
wrote row 15
wrote row 16
wrote row 17
wrote row 18
wrote row 19
wrote row 20
wrote row 21
wrote row 22
wrote row 23
wrote row 24
wrote row 25
wrote row 26
wrote row 27
wrote row 28
wrote row 29
wrote row 30
wrote row 31
wrote row 32
wrote row 33
wrote row 34
wrote row 35
wrote row 36
wrote row 37
wrote row 38
wrote row 39
wrote row 40
wrote row 41
wrote row 42
wrote row 43
wrote row 44
wrote row 45
wrote row 46
wrote row 47
wrote row 48
wrote row 49
wrote row 50
wrote row 51
wrote row 52
wrote row 53
wrote row 54
wrote row 55
wrote row 56
wrote row 57
wrote row 58
wrote row 59
wrote row 60
wrote row 61
wrote row 62
wrote row 63
wrote row 64
wrote row 65
wrote row 66
wrote row 67
wrote row 68
wrote row 69
wrote row 70
wrote row 71
wrote row 72
wrote row 73
wrote row 74
wrote row 75
wrote row 76
wrote row

In [9]:
results_df

,Full Text,Predicate,ID subgroups,Best response,Exact Match,Phrase Cosine Sim.,Sent. Cosine Sim.,Extraction Outcome
0,"LGBTQ+ people are abnormal, crazy, have family...",abnormal,LGBTQ,are abnormal,0,0.82795,0.989485,correct
1,"LGBTQ+ people are abnormal, crazy, have family...",crazy,LGBTQ,are crazy,0,0.647407,0.987745,correct
2,"LGBTQ+ people are abnormal, crazy, have family...",have family issues,LGBTQ,have family issues,1,1.0,1.0,correct
3,"LGBTQ+ people are abnormal, crazy, have family...",have health issues,LGBTQ,have health issues,1,1.0,1.0,correct
4,Unfit for professional work,unfit for professional work,LGBTQ,struggle with mental health,0,0.237125,0.575729,hallucination
...,...,...,...,...,...,...,...,...
273,slurs/ being treated like there's something wr...,there is something wrong,Pansexual,All pansexual people are abnormal.,0,0.284982,0.708158,grammar
274,Being undermined in an academic environment,undermined in an academic environment,Pansexual,All pansexual people struggle.,0,0.097492,0.548966,hallucination
275,"More willing to have threesomes, ""easy"", bigge...",more willing to have threesomes,Pansexual,More likely to engage in threesomes.,0,0.792879,0.947729,hallucination
276,"More willing to have threesomes, ""easy"", bigge...",easy,Pansexual,cheat on their partners,0,0.228233,0.518241,correct


In [10]:
results_df.groupby(["ID subgroups"])['Phrase Cosine Sim.'].mean()

ID subgroups
Asexual        0.397418
Bisexual       0.428189
Gay            0.520412
LGBTQ            0.5372
Lesbian        0.416833
Nonbinary      0.512742
Pansexual      0.393002
Queer          0.466635
Transgender     0.44074
Name: Phrase Cosine Sim., dtype: object

In [11]:
results_df['Phrase Cosine Sim.'].mean()

0.4688638756118661

In [47]:
results_df.groupby(["ID subgroups"])['Sent. Cosine Sim.'].mean()

ID subgroups
Asexual        0.740199
Bisexual       0.755286
Gay            0.825764
LGBTQ          0.835171
Lesbian        0.744472
Nonbinary      0.819037
Pansexual      0.706633
Queer          0.786981
Transgender    0.749971
Name: Sent. Cosine Sim., dtype: object

In [48]:
results_df['Sent. Cosine Sim.'].mean()

0.7836801141714879

In [49]:
results_df.groupby(["ID subgroups"])['Exact Match'].value_counts()

ID subgroups  Exact Match
Asexual       0              24
Bisexual      0              40
              1               2
Gay           0              53
              1               3
LGBTQ         0              33
              1               3
Lesbian       0              16
              1               1
Nonbinary     0              24
              1               3
Pansexual     0              17
Queer         0              26
              1               2
Transgender   0              30
              1               1
Name: count, dtype: int64

In [50]:
results_df['Exact Match'].value_counts()

Exact Match
0    263
1     15
Name: count, dtype: int64

In [10]:
results_df.groupby(["ID subgroups"])['Extraction Outcome'].value_counts(normalize=True)

ID subgroups  Extraction Outcome
Asexual       grammar               0.500000
              hallucination         0.250000
              correct               0.125000
              opposite              0.083333
              other                 0.041667
Bisexual      hallucination         0.428571
              correct               0.309524
              grammar               0.214286
              other                 0.023810
              opposite              0.023810
Gay           correct               0.571429
              hallucination         0.196429
              grammar               0.196429
              other                 0.017857
              opposite              0.017857
LGBTQ         correct               0.527778
              hallucination         0.305556
              opposite              0.083333
              other                 0.055556
              grammar               0.027778
Lesbian       correct               0.352941
              opposite

In [11]:
results_df['Extraction Outcome'].value_counts(normalize=True)

Extraction Outcome
correct          0.381295
hallucination    0.312950
grammar          0.215827
opposite         0.061151
other            0.028777
Name: proportion, dtype: float64

In [32]:
results_df[(results_df['Extraction Outcome'] == 'hallucination') & (results_df['ID subgroups'] == 'Gay')]

,Full Text,Predicate,ID subgroups,Best response,Exact Match,Phrase Cosine Sim.,Sent. Cosine Sim.,Extraction Outcome
96,Easily fall in love with straight guys and hav...,Easily fall in love with straight guys and hav...,Gay,are promiscuous,0,0.436906,0.644943,hallucination
100,"Homophobia, monkeypox, hiv. Dirty",have monkeypox,Gay,are diseased,0,0.337342,0.669088,hallucination
107,"There's the stigma that the gay community is ""...",pedophile,Gay,are promiscuous.,0,0.367087,0.693452,hallucination
108,"There's the stigma that the gay community is ""...",groomer,Gay,are flamboyant.,0,0.141368,0.605898,hallucination
109,I think the big one that I usually hear is des...,bad,Gay,are promiscuous.,0,0.15583,0.579438,hallucination
114,(1) hypersexual (2) love hard drugs (3) just p...,not serious about career,Gay,are carefree,0,0.127956,0.592248,hallucination
116,"Not ""normal"", must be effeminate",not normal,Gay,are promiscuous,0,0.167817,0.600583,hallucination
121,Religious sinful mentalities,sinful,Gay,All gay people engage in promiscuous behavior.,0,0.263079,0.656176,hallucination
130,Both that we're post-sexual as well as creepy ...,post-sexual,Gay,All queer people are post-sexual.,0,0.646741,0.945093,hallucination
131,Both that we're post-sexual as well as creepy ...,creepy,Gay,All queer people are creepy.,0,0.510929,0.938331,hallucination
